In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'

In [3]:
cap = cv2.VideoCapture('mats/vid1_IR.avi')
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    G = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    cv2.imshow('IR', G)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

In [15]:
def process_frame(current_frame: np.ndarray) -> np.ndarray:
    original_image: np.ndarray = current_frame.copy()
    
    gray_image: np.ndarray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
    
    _, threshold_image = cv2.threshold(gray_image, 45, 255, cv2.THRESH_BINARY)
    median: np.ndarray = cv2.medianBlur(threshold_image, 7)
    # rozmycie medianowe
    dilated = cv2.dilate(median, np.ones((5, 5), np.uint8))
    closed: np.ndarray = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, np.ones((7, 7), np.uint8), iterations=2)
    

    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(closed)



    objects_on_screen = num_labels > 1
    if objects_on_screen:
        skipped_objects = 0
        for object_index in range(1, num_labels):
            # disregard objects with height less than 200, width less than 200, or area less than 400
            x, y, w, h, area = stats[object_index]
            
            too_small: bool = area < 400
            not_rectangle: bool = abs(w - h) < 50
            too_narrow: bool = h < 100
            if too_small or not_rectangle or too_narrow:
                skipped_objects += 1
                continue
                
            centroid_x, centroid_y = centroids[object_index]
            
            cv2.rectangle(original_image, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(original_image, str(object_index - skipped_objects), (int(centroid_x), int(centroid_y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1)
    
    return original_image

def process_video(file: str, save = False) -> None:
    capture = cv2.VideoCapture(file)
    if save:
        frames = []
    while capture.isOpened():
        has_next_frame, current_frame = capture.read()
        if not has_next_frame:
            break
    
        new_image = process_frame(current_frame)
        if save:
            frames.append(new_image)
        cv2.imshow('IR', new_image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    if save:
        # Save the video
        print('Saving video...')
        height, width, layers = frames[0].shape
        size = (width, height)
        out = cv2.VideoWriter('output2.avi', cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
        for i in range(len(frames)):
            out.write(frames[i])
        out.release()
        print('Video saved')
        
    capture.release()
    cv2.destroyAllWindows()

process_video('mats/vid1_IR.avi', save = True)

ciocia gada moment

